In [1]:
import matplotlib.pyplot as plt


import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, MaxPooling2D, BatchNormalization, \
SpatialDropout2D, GlobalMaxPooling2D, Input, LeakyReLU, AveragePooling2D, MaxPool2D, Concatenate, Lambda
from tensorflow.keras.utils import Sequence
import pandas as pd
import numpy as np
import os
import random
import cv2

import pickle

INPUT_FOLDER = 'C:/Users/alexa/___Licenta_v2/dataset/train/'
OUTPUT_FOLDER = 'D:/Licenta/new_try_kaggle/cnn_models/'
OUTPUT_FOLDER_TB = 'D:/Licenta/new_try_kaggle/cnn_models/tensor_boards/'

train_samples = os.listdir(INPUT_FOLDER)
train_samples.sort()

random.shuffle(train_samples)

labels_df = pd.read_csv('D:/Licenta/new_try_kaggle/train_labels.csv', index_col=0)
labels_df['label'] = labels_df['label'].astype(object) 

In [2]:
partition = dict()
all_labels = dict()
partition['train'] = train_samples[:100000]
partition['validate'] = train_samples[100000:]

all_cancer = list()

for sample in train_samples:
    
    label = labels_df.get_value(sample[:-4], 'label')
    all_labels[sample] = label
      
cancer = 0
non_cancer = 0
for key in partition['validate']:
    if all_labels[key] == 1:
        cancer+=1
    else:
        non_cancer+=1
        
print(f"cancer-{repr(cancer)}-non_cancer-{repr(non_cancer)}")
cancer = 0
non_cancer = 0
for key in partition['train']:
    if all_labels[key] == 1:
        cancer+=1
    else:
        non_cancer+=1
        
print(f"cancer-{repr(cancer)}-non_cancer-{repr(non_cancer)}")

C:\Users\alexa\AppData\Local\conda\conda\envs\licentav2\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


cancer-3838-non_cancer-5582
cancer-40487-non_cancer-59513


In [3]:
def get_model(kernel_size, pool_size, first_filters, second_filters, third_filters, fourth_filters, dropout_conv,dropout_dense):
    
    #now add layers to it
    Input_1 = Input(shape=(32,32,4))
    x_rgb = Lambda(lambda Input_1 : Input_1[:,:,:,:3])(Input_1)
    print(x_rgb.shape)

    x_grey = Lambda(lambda Input_1 : Input_1[:,:,:,3:])(Input_1)
    #conv block 1 for the first three channels
    first_Convolution2D_1_2 = Conv2D(first_filters, (3,3), use_bias=False)(x_rgb)
    
    first_BatchNormalization_1_2 = BatchNormalization()(first_Convolution2D_1_2)
    first_Activation_1_2 = Activation('relu')(first_BatchNormalization_1_2)
    
    first_SpatialDroupout2D_1 = SpatialDropout2D(dropout_conv)(first_Activation_1_2)
    
    #conv block 2 for the first three channels
    first_Convolution2D_2_1 = Conv2D(second_filters, kernel_size, use_bias=False)(first_SpatialDroupout2D_1)
    first_BatchNormalization_2_1 = BatchNormalization()(first_Convolution2D_2_1)
    
    first_Activation_2_1 = Activation('relu')(first_BatchNormalization_2_1)
    first_Convolution2D_2_2 = Conv2D(second_filters, kernel_size, use_bias=False)(first_Activation_2_1)
    
    first_BatchNormalization_2_2 = BatchNormalization()(first_Convolution2D_2_2)
    first_Activation_2_2 = Activation('relu')(first_BatchNormalization_2_2)
    
    first_SpatialDroupout2D_2 = SpatialDropout2D(dropout_conv)(first_Activation_2_2)
    
    #conv block 3 for the first three channels
    first_Convolution2D_3_1 = Conv2D(third_filters, kernel_size, use_bias=False)(first_SpatialDroupout2D_2)
    first_BatchNormalization_3_1 = BatchNormalization()(first_Convolution2D_3_1)
    
    first_Activation_3_1 = Activation('relu')(first_BatchNormalization_3_1)
    first_Convolution2D_3_2 = Conv2D(third_filters, kernel_size, use_bias=False)(first_Activation_3_1)
    
    first_BatchNormalization_3_2 = BatchNormalization()(first_Convolution2D_3_2)
    first_Activation_3_2 = Activation('relu')(first_BatchNormalization_3_2)
    
    first_SpatialDroupout2D_3 = SpatialDropout2D(dropout_conv)(first_Activation_3_2)
    
    #conv block 4 for the first three channels
    first_Convolution2D_4_1 = Conv2D(fourth_filters, kernel_size, use_bias=False)(first_SpatialDroupout2D_3)
    first_BatchNormalization_4_1 = BatchNormalization()(first_Convolution2D_4_1)
    
    first_Activation_4_1 = Activation('relu')(first_BatchNormalization_4_1)
    first_Convolution2D_4_2 = Conv2D(fourth_filters, kernel_size, use_bias=False)(first_Activation_4_1)
    
    first_BatchNormalization_4_2 = BatchNormalization()(first_Convolution2D_4_2)
    first_Activation_4_2 = Activation('relu')(first_BatchNormalization_4_2)
    
    first_SpatialDroupout2D_4 = SpatialDropout2D(dropout_conv)(first_Activation_4_2)
    first_Flatten1 = Flatten()(first_SpatialDroupout2D_4)
    
    first_Dense_1 = Dense(200, use_bias = False)(first_Flatten1)
    print(first_Dense_1.shape)
    #first_MaxPool = MaxPooling2D(pool_size)(first_SpatialDroupout2D_4)
    
    #conv block 1 for the fourth channel
    fourth_Convolution2D_1_2 = Conv2D(first_filters, (3,3), use_bias=False)(x_grey)
    
    fourth_BatchNormalization_1_2 = BatchNormalization()(fourth_Convolution2D_1_2)
    fourth_Activation_1_2 = Activation('relu')(fourth_BatchNormalization_1_2)
    
    fourth_SpatialDroupout2D_1 = SpatialDropout2D(dropout_conv)(fourth_Activation_1_2)
    
    #conv block 2 for the fourth channel
    fourth_Convolution2D_2_1 = Conv2D(second_filters, kernel_size, use_bias=False)(fourth_SpatialDroupout2D_1)
    fourth_BatchNormalization_2_1 = BatchNormalization()(fourth_Convolution2D_2_1)
    
    fourth_Activation_2_1 = Activation('relu')(fourth_BatchNormalization_2_1)
    fourth_Convolution2D_2_2 = Conv2D(second_filters, kernel_size, use_bias=False)(fourth_Activation_2_1)
    
    fourth_BatchNormalization_2_2 = BatchNormalization()(fourth_Convolution2D_2_2)
    fourth_Activation_2_2 = Activation('relu')(fourth_BatchNormalization_2_2)
    
    fourth_SpatialDroupout2D_2 = SpatialDropout2D(dropout_conv)(fourth_Activation_2_2)
    
    #conv block 3 for the fourth channel
    fourth_Convolution2D_3_1 = Conv2D(third_filters, kernel_size, use_bias=False)(fourth_SpatialDroupout2D_2)
    fourth_BatchNormalization_3_1 = BatchNormalization()(fourth_Convolution2D_3_1)
    
    fourth_Activation_3_1 = Activation('relu')(fourth_BatchNormalization_3_1)
    fourth_Convolution2D_3_2 = Conv2D(third_filters, kernel_size, use_bias=False)(fourth_Activation_3_1)
    
    fourth_BatchNormalization_3_2 = BatchNormalization()(fourth_Convolution2D_3_2)
    fourth_Activation_3_2 = Activation('relu')(fourth_BatchNormalization_3_2)
    
    fourth_SpatialDroupout2D_3 = SpatialDropout2D(dropout_conv)(fourth_Activation_3_2)
    
    #conv block 4 for the fourth channel
    fourth_Convolution2D_4_1 = Conv2D(fourth_filters, kernel_size, use_bias=False)(fourth_SpatialDroupout2D_3)
    fourth_BatchNormalization_4_1 = BatchNormalization()(fourth_Convolution2D_4_1)
    
    fourth_Activation_4_1 = Activation('relu')(fourth_BatchNormalization_4_1)
    fourth_Convolution2D_4_2 = Conv2D(fourth_filters, kernel_size, use_bias=False)(fourth_Activation_4_1)
    
    fourth_BatchNormalization_4_2 = BatchNormalization()(fourth_Convolution2D_4_2)
    fourth_Activation_4_2 = Activation('relu')(fourth_BatchNormalization_4_2)
    
    fourth_SpatialDroupout2D_4 = SpatialDropout2D(dropout_conv)(fourth_Activation_4_2)
    fourth_Flatten1 = Flatten()(fourth_SpatialDroupout2D_4)
    
    fourth_Dense_1 = Dense(200, use_bias = False)(fourth_Flatten1)
    print(fourth_Dense_1.shape)
    #fourth_MaxPool = MaxPooling2D(pool_size)(fourth_SpatialDroupout2D_4)
    
    concat_layers = tf.keras.layers.concatenate([first_Dense_1,fourth_Dense_1], axis = 1)
    
    
    
    #a fully connected (also called dense) layer at the end
    Dropout_after_concat = Dropout(dropout_dense)(concat_layers)
    Dense_1 = Dense(50, use_bias = False)(Dropout_after_concat)
    BatchNormalization_4_1 = BatchNormalization()(Dense_1)
    Activation_4_1 = Activation('relu')(BatchNormalization_4_1)
    Dropout_1 = Dropout(dropout_dense)(Activation_4_1)
    
    #Dense_2 = Dense(100, use_bias = False)(Dropout_1)
    #BatchNormalization_5_1 = BatchNormalization()(Dense_2)
    #Activation_5_1 = Activation('relu')(BatchNormalization_5_1)
    #Dropout_2 = Dropout(dropout_dense)(Activation_5_1)
    
    Dense_2 = Dense(2, activation= 'softmax')(Dropout_1)
    
    #finally convert to values of 0 to 1 using the sigmoid activation function
    return Model([Input_1], [Dense_2])


In [4]:
#batch generator to load the data in batches

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=10, dim=(32, 32), n_channels=4,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            temp_sample = np.empty((*self.dim, self.n_channels))
            temp_sample_greyscale = np.empty((*self.dim, self.n_channels))
            
            rotate_factor = random.randint(0,3)
            
            temp_sample_rgb = cv2.imread(os.path.join(INPUT_FOLDER, ID))
            temp_sample_rgb = temp_sample_rgb[32:64,32:64,]
            
            temp_sample_greyscale = cv2.imread(os.path.join(INPUT_FOLDER, ID), cv2.IMREAD_GRAYSCALE)
            temp_sample_greyscale = temp_sample_greyscale[32:64,32:64]
            
            temp_sample_rgb = np.rot90(temp_sample_rgb, rotate_factor)
            temp_sample_greyscale = np.rot90(temp_sample_greyscale, rotate_factor)
            #zero center each channel
            temp_sample[:,:,0] = (temp_sample[:,:,0] - 177.36477546559416)
            temp_sample[:,:,1] = (temp_sample[:,:,1] - 138.8431052117129)
            temp_sample[:,:,2] = (temp_sample[:,:,2] - 179.00444484090832)
            temp_sample[:,:,3] = (temp_sample_greyscale - 155.25222880621973)
            
            X[i,] = temp_sample

            # Store class
            y[i] = self.labels[ID]

        return X, to_categorical(y, num_classes=self.n_classes)

In [ ]:
kernel_size = (2,2)
pool_size= (2,2)
first_filters = 64
second_filters = 64
third_filters = 128
fourth_filters = 192

#dropout is used for regularization here with a probability of 0.35 for conv layers, 0.5 for the dense layer at the end
dropout_conv = 0.35
dropout_dense = 0.5

In [ ]:
my_training_batch_generator = DataGenerator(partition['train'], labels = all_labels, batch_size = 200)
my_validation_batch_generator = DataGenerator(partition['validate'], labels = all_labels, batch_size = 157)

In [ ]:
cnn_name = 'rgb_grayscale_split_with_image_roatation_reworked_input'
print(cnn_name)

rgb_grayscale_split_with_image_roatation_reworked_input


In [ ]:
TB= TensorBoard(log_dir=OUTPUT_FOLDER_TB +cnn_name)

model = get_model(kernel_size, pool_size, first_filters, second_filters, third_filters, fourth_filters, dropout_conv, dropout_dense)

model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['binary_accuracy'])

model.summary()

class_weight = {0: 1.,
                1: 1.25
    
}

model.fit_generator(generator=my_training_batch_generator,
                                          steps_per_epoch=(500),
                                          epochs=15,
                                          verbose=1,
                                          validation_data=my_validation_batch_generator,
                                          validation_steps=(60),
                                          callbacks = [TB],
                                          class_weight = class_weight)




model.save(OUTPUT_FOLDER + cnn_name+'.h5')

(?, 32, 32, 3)
(?, 200)
(?, 200)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 4)    0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 32, 32, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 32, 32, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 30, 30, 64)   1728        lambda[0][0]                     
____________________________________________________________________________

Epoch 1/15


C:\Users\alexa\AppData\Local\conda\conda\envs\licentav2\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in subtract
C:\Users\alexa\AppData\Local\conda\conda\envs\licentav2\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in subtract
C:\Users\alexa\AppData\Local\conda\conda\envs\licentav2\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in subtract


500/500 [==============================] - 184s 368ms/step - loss: 0.5734 - binary_accuracy: 0.7294 - val_loss: 0.8022 - val_binary_accuracy: 0.5953
Epoch 2/15
500/500 [==============================] - 182s 364ms/step - loss: 0.5330 - binary_accuracy: 0.7535 - val_loss: 0.6671 - val_binary_accuracy: 0.6453
Epoch 3/15
500/500 [==============================] - 181s 362ms/step - loss: 0.5235 - binary_accuracy: 0.7579 - val_loss: 0.6436 - val_binary_accuracy: 0.6574
Epoch 4/15
470/500 [===========================>..] - ETA: 10s - loss: 0.5218 - binary_accuracy: 0.7598